In [5]:
import os
from openai import OpenAI
from dotenv import load_dotenv

def ask_chat_gpt(prompt):
    # Load environment variables from the .env file
    load_dotenv()

    # Access the API key from the environment
    api_key = os.getenv("OPENAI_KEY")

    client = OpenAI(
        # This is the default and can be omitted
        api_key=api_key,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o",
    )
    message = chat_completion.choices[0].message.content
    return message

In [6]:
def make_gpt_promt(filename):
    start_promt = """
    Generate LaTeX code from the provided Python code. Follow these guidelines:

    1. **Document Structure**:
    - Start with `\\section{}` for the algorithm title.
    - Add a `\\subsection{Introduction}` for a brief overview of the algorithm.
    - Include a `\\subsection{Variables}` section that lists all variables with descriptions, using subscript notation (e.g., `v_{\\text{earth}}`).
    - Add a `\\subsection{Formulas}` section presenting the code's logic as LaTeX formulas. Use subscripted symbols for variable names instead of copying Python variable names directly.

    2. **Formatting Rules**:
    - Ensure that the output includes **only** the LaTeX content, without `\\documentclass`, `\\usepackage`, `\\begin{document}`, `\\end{document}`, or any unrelated text.
    - Do **not** include the triple backticks (e.g., ```latex or ```).
    - Properly close all LaTeX environments (e.g., `\\begin{align*}...\\end{align*}`).
    - Ensure all brackets, parentheses, and braces are matched correctly.
    - Maintain consistent subscript notation for all variables.

    3. **Important Notes**:
    - **Do not** include any text or explanations outside the LaTeX code.
    - Only the relevant LaTeX content for the `\\section`, `\\subsection`, `\\begin{align*}`, and `\\end{align*}` parts should be generated.
    - Ensure no extra or unrelated LaTeX sections are added.
    """

    file_path = f'python_code/{filename}'

    # Open the file in read mode and save its contents to a string
    with open(file_path, 'r') as file:
        file_contents = file.read()

    total_prompt = start_promt + file_contents
    print("Finishin making prompt")
    return total_prompt

def  make_algo_doc(filename, name):
    total_prompt = make_gpt_promt(filename)    
    print("Asking gpt...")
    message = ask_chat_gpt(total_prompt)
    folder_path = "algo_docs/"
    document_path_name = folder_path + name + "_doc.tex"
    with open(document_path_name, "w") as file:
        file.write(message)
    print("Finish making algo doc")

In [10]:
from pylatex import Document, Section, Chapter, Subsection
from pylatex.utils import NoEscape
import subprocess

code_names = ["calculate_hohmann_transfer", "gravitational_assist"]
# Create and structure the LaTeX document programmatically
doc = Document(documentclass="report")

# Include custom preamble and metadata files
doc.preamble.append(NoEscape(r'\input{algo_docs/init.tex}'))  # Preamble with packages and styles
doc.append(NoEscape(r'\input{algo_docs/title_page.tex}'))     # Title page and table of contents
doc.append(NoEscape(r'\tableofcontents'))

# Create content sections
with doc.create(Chapter('Introduction')):
    doc.append('This document provides an overview of various algorithms and their significance in computational problem-solving. It aims to set the foundation for understanding the design, analysis, and application of different algorithms across various domains.')

with doc.create(Chapter('Algorithms')):
    doc.append('This chapter presents detailed analyses of different algorithms. It explores their mechanisms, use cases, and the theoretical concepts that underlie their effectiveness. Each algorithm is discussed with practical insights to highlight its strengths and potential limitations.')

# Add additional content with sub-chapters for the code
for filename in os.listdir('python_code/'):
    name = filename.replace(".py","")
    folder_path = "algo_docs/"
    document_path_name = folder_path + name + "_doc.tex"
    print(f"Running for {filename}")
    #make_algo_doc(filename, name)

    python_code_path = f"python_code/{filename}"
    doc.append(NoEscape(rf'\input{{{document_path_name}}}'))
        
    # Add sub-chapter for the code listing
    with doc.create(Subsection('Code Listing')):
        doc.append(NoEscape(rf'\lstinputlisting[language=Python]{{{python_code_path}}}'))

    # Generate the .tex file
    doc.generate_tex("programmatic_report")

    # Compile to PDF
    subprocess.run(["pdflatex", "programmatic_report.tex"])

Running for optimize_trajectory.py
This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./programmatic_report.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-01-16>
(/usr/share/texlive/texmf-dist/tex/latex/base/report.cls
Document Class: report 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf/tex/latex/lm/lmodern.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty)
(/usr/share/texlive/texmf-dist/tex/latex/lastpage/lastpage.sty)
(./algo_docs/init.tex
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/share/texlive/texmf-dist/tex/latex/

In [ ]:
for filename in os.listdir('python_code/'):
    name = filename.replace(".py","")
    folder_path = "algo_docs/"
    document_path_name = folder_path + name + "_doc.tex"
    print(f"Running for {filename}")
    #make_algo_doc(filename, name)

This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./programmatic_report.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-01-16>
(/usr/share/texlive/texmf-dist/tex/latex/base/report.cls
Document Class: report 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/fontenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf/tex/latex/lm/lmodern.sty)
(/usr/share/texlive/texmf-dist/tex/latex/base/textcomp.sty)
(/usr/share/texlive/texmf-dist/tex/latex/lastpage/lastpage.sty)
(./algo_docs/init.tex
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsgen.sty))
(/usr/share/te

CompletedProcess(args=['pdflatex', 'programmatic_report.tex'], returncode=0)